# Data cleaning 1

See comments and explanations below

In [172]:
# import modules
import numpy as np
import pandas as pd
import scipy as sp
import math
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import tree
from sklearn import ensemble
from sklearn.cross_validation import KFold
import csv
import StringIO
import string

In [173]:
# load data
full = pd.read_csv('full_data_2.csv')
full.columns

Index([u'Unnamed: 0', u'Inspection ID', u'DBA Name', u'AKA Name', u'License #',
       u'Facility Type', u'Risk', u'Address', u'City', u'State', u'Zip',
       u'Inspection Date', u'Inspection Type', u'Results', u'Violations',
       u'Latitude', u'Longitude', u'Location', u'Day of Week', u'Month',
       u'Community Area', u'num_crimes_comm_area', u'num_complaints_comm_area',
       u'COMMUNITY AREA NAME', u'PERCENT OF HOUSING CROWDED',
       u'PERCENT HOUSEHOLDS BELOW POVERTY', u'PERCENT AGED 16+ UNEMPLOYED',
       u'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       u'PERCENT AGED UNDER 18 OR OVER 64', u'PER CAPITA INCOME ',
       u'HARDSHIP INDEX', u'Birth Rate', u'General Fertility Rate',
       u'Low Birth Weight', u'Prenatal Care Beginning in First Trimester',
       u'Preterm Births', u'Teen Birth Rate', u'Assault (Homicide)',
       u'Breast cancer in females', u'Cancer (All Sites)',
       u'Colorectal Cancer', u'Diabetes-related', u'Firearm-related',
       u'Infant Mo

In [174]:
# get dataframe with canvass inspections and complaints only
full_2 = full[(full['Inspection Type'] == 'CANVAS') | (full['Inspection Type'] == 'Canvass') | \
                 (full['Inspection Type'] == 'Complaint')]

For this project, we were interested in ranking restaurants based on their likelihoods to fail regular canvass inspections. We used complaints against restaurants as data to inform us about these likelihoods. Thus, we deleted all inspections with inspection types that were neither regular canvass inspections nor complaints.

In [182]:
# if AKA name is missing, impute the DBA name
for i in range(len(full_2)):
    if pd.isnull(full_2.iloc[i, 3]):
        full_2.iloc[i, 3] = full_2.iloc[i, 2]

In [183]:
# due to punctuation inconsistencies in restaurant names with commas and apostrophes, we deleted punctuation in names that
# contained them
for i in range(len(full_2)):
    if i % 1000 == 0:
        print i,
    if ',' in full_2.iloc[i, 2] or ',' in full_2.iloc[i, 3]:
        full_2.iloc[i, 2] = full_2.iloc[i, 3].translate(None, string.punctuation)
        full_2.iloc[i, 3] = full_2.iloc[i, 3].translate(None, string.punctuation)
    if '\'' in full_2.iloc[i, 2] or '\'' in full_2.iloc[i, 3]:
        full_2.iloc[i, 2] = full_2.iloc[i, 3].translate(None, string.punctuation)
        full_2.iloc[i, 3] = full_2.iloc[i, 3].translate(None, string.punctuation)

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000


In [197]:
#Sort restaurants by address, AKA name, License #, and DBA name
full_2_sort = full_2.sort_values(by = ['Address', 'AKA Name', 'License #', 'DBA Name'])
full_2_sort.reset_index(drop = True, inplace = True)

In [198]:
# give unique identifiers to restaurants
restaurant_identifier = []
number = 1

for i in range(len(full_2_sort)):
    
    #Give restaurant id = number
    restaurant_identifier.append(number)
    
    #If we get to the end, break loop
    if i == len(full_2_sort) - 1:
        break

    #If the next restaurant has the same name and address, continue to next interation of loop 
    #i.e. give them the same identifier
    if (full_2_sort.loc[i, 'DBA Name'][0:20] == full_2_sort.loc[i + 1, 'DBA Name'][0:20] and 
        full_2_sort.loc[i, 'Address'][0:10] == full_2_sort.loc[i + 1, 'Address'][0:10] and 
        full_2_sort.loc[i, 'AKA Name'][0:6] == full_2_sort.loc[i + 1, 'AKA Name'][0:6]):
        continue
    
    #Otherwise, increase the identifier by 1
    else:
        number += 1

full_3 = full_2_sort.copy()
full_3['restaurant_id'] = pd.Series(restaurant_identifier, index = full_3.index)

In [199]:
# drop observations with no community area available
full_4 = full_3.dropna(subset = [full_3.columns[21]])
full_4 = full_4.drop(['Unnamed: 0'], axis = 1)

In [202]:
# make text denoting regular canvass inspections consistent
for i in range(len(full_4)):
    if full_4.iloc[i, 11] == 'CANVAS':
        full_4.iloc[i, 11] = 'Canvass'

In [203]:
# change inspection results with 'Pass w/ Condtions' to 'Pass'
for i in range(len(full_4)):
    if full_4.iloc[i, 12] == 'Pass w/ Conditions':
        full_4.iloc[i, 12] = 'Pass'

Having three classes for inspection results was unnecessary for this project since our goal was to rank restaurants by likelihood of failure. Thus, we decided to mark inspections with the result 'Pass w/ Conditions' as 'Pass' since those inspections were ultimately passes.